# Notebook Instructions

1. If you are new to Jupyter notebooks, please go through this introductory manual <a href='https://quantra.quantinsti.com/quantra-notebook' target="_blank">here</a>.
1. Any changes made in this notebook would be lost after you close the browser window. **You can download the notebook to save your work on your PC.**
1. Before running this notebook on your local PC:<br>
i.  You need to set up a Python environment and the relevant packages on your local PC. To do so, go through the section on "**Run Codes Locally on Your Machine**" in the course.<br>
ii. You need to **download the zip file available in the last unit** of this course. The zip file contains the data files and/or python modules that might be required to run this notebook.

## Assemble States

In this notebook, you will learn how to create input features to construct a state. You have already learned to make candlestick bars of 5 minutes, 1 hour, 1 day for a specific lookback period. Using these bars of different granularity, you will create input features and construct the state.

The steps to do the same are as follows:
1. [Get last N time bars](#timebars)
2. [Construct state](#state)<br>
    2.1. [Create stationary candlestick bars](#candlesticks)<br>
    2.2. [Create technical indicators](#indicators)<br>
    2.3. [Create time signature](#time)

<a id='timebars'></a> 
## Get last N time bars

You already have done this step in the previous notebook. You need to create the last N time bars from the current time to create input features. Here we will define a current time. In the Game class, the current time is fetched using the current index of the price data to get the last N time bars.

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import datetime
import talib

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# The data is stored in the directory 'data_modules'
path = '../data_modules/'

# Read the pickle file
bars5m = pd.read_pickle(path+ 'PriceData5m.bz2')

ohlcv_dict = {
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    }

bars1h = bars5m.resample('1H', closed='right', label='left').agg(ohlcv_dict).dropna()
bars1d = bars5m.resample('1D', closed='right', label='left').agg(ohlcv_dict).dropna()

def get_last_N_timebars(bars5m, bars1h, bars1d, curr_time, lkbk):
    '''This function gets the timebars for the 5m, 1hr and 1d resolution based
    on the lookback we've specified.
    '''
   
    """Width of the 5m, 1hr, and 1d"""
    wdw5m = 9
    wdw1h = np.ceil(lkbk*15/24.)
    wdw1d = np.ceil(lkbk*15)

    """Creating the timebars based on the lookback"""
    last5m = bars5m[curr_time-timedelta(wdw5m):curr_time].iloc[-lkbk:]
    last1h = bars1h[curr_time-timedelta(wdw1h):curr_time].iloc[-lkbk:]
    last1d = bars1d[curr_time-timedelta(wdw1d):curr_time].iloc[-lkbk:]
    
    return last5m, last1h, last1d

In [2]:
# Set the current time
curr_time = datetime.datetime(2012, 5, 10, 11, 0, 0)

# Fixed the lookback period to get last time bars from the current time.
# This is also used to calculate the value of technical indicators. This is a hyperparameter that you can tweak.
lkbk = 20

# Store the last N time bars for 5mins, 1h, and 1d bars
last5m, last1h, last1d = get_last_N_timebars(
    bars5m, bars1h, bars1d, curr_time, lkbk)

last5m

,open,high,low,close,volume
Time,,,,,
2012-05-09 15:55:00-04:00,116.109,116.113,115.810,115.870,3311517.0
2012-05-09 16:00:00-04:00,115.879,115.938,115.742,115.827,5225624.0
2012-05-10 09:35:00-04:00,116.638,116.749,116.578,116.749,3081260.0
2012-05-10 09:40:00-04:00,116.741,116.741,116.596,116.689,2404169.0
2012-05-10 09:45:00-04:00,116.698,116.698,116.408,116.502,2932492.0
2012-05-10 09:50:00-04:00,116.502,116.689,116.450,116.681,2169711.0
2012-05-10 09:55:00-04:00,116.681,116.783,116.621,116.664,2074586.0
2012-05-10 10:00:00-04:00,116.664,116.758,116.578,116.587,1836284.0
2012-05-10 10:05:00-04:00,116.587,116.655,116.510,116.536,2586152.0


The above output shows the last 20 (lookback) data points from the current time for the 5 mins bar. You can also check the result for 1 hour and 1 day bars.

<a id='state'></a> 

## Construct state

In this step, you will create input features and append them to the state array. First, you will create an array `state` which will store all the input features. The first input feature is candlestick bars for the 5 mins, 1 hour, and 1 day data. Next, you will add technical indicators for each granularity of data. Finally, you will add the time of day, day of the week and position as features to the state array. 

1. Candlestick bars
2. Technical indicators
3. Time signatures

But before calculating input features, you need to convert dataframe into the 1-dimensional array as the neural networks prefer array as an input.

In [3]:
"""Adding State Variable"""
state = np.array([])

In [4]:
state.shape

(0,)

We start with a state array of size 0.

<a id='flatten'></a> 
### Flatten the dataframe

You will flatten the dataframe into the 1-dimensional array. This will be done using the `flatten()` method of Numpy.

Syntax: 
```python
DataFrame.flatten()
```
Return:
    An array flattened to one dimension

In [5]:
last5m_flat = last5m.iloc[-10:, :-1].values.flatten()
last5m_flat

array([116.638, 116.672, 116.416, 116.425, 116.425, 116.604, 116.399,
       116.485, 116.485, 116.578, 116.263, 116.298, 116.297, 116.314,
       116.211, 116.254, 116.254, 116.322, 116.126, 116.254, 116.254,
       116.339, 116.237, 116.314, 116.314, 116.425, 116.297, 116.374,
       116.365, 116.374, 116.237, 116.289, 116.297, 116.297, 116.143,
       116.16 , 116.152, 116.169, 115.99 , 116.16 ])

In [6]:
print('Shape of the array:', last5m_flat.shape)

Shape of the array: (40,)


You can see the 5 mins bar is flatten into an array of size 40 as there are 4 columns (open, high, low, close) and 10 rows in last5m dataframe.

<a id='candlesticks'></a> 
## Create stationary candlestick bars

The first input feature is candlestick bars. As you have already learned in the previous video that candlestick bars are not stationary and neural network prefer stationary features. Therefore, you have to make candlestick bars stationary. This is done using the z-score.

**Z-score = (price bars - mean of bars)/ standard deviation of bars**

Mean and standard deviation are calculated by simple `mean` and `std` function of NumPy.

In [7]:
"""Normalizing candlesticks for 5 mins data"""
last5m_norm = (last5m_flat-np.mean(last5m_flat))/np.std(last5m_flat)
last5m_norm

array([ 2.21143692,  2.44988568,  0.65450678,  0.71762557,  0.71762557,
        1.97298816,  0.5352824 ,  1.1384175 ,  1.1384175 ,  1.79064499,
       -0.41851264, -0.17305068, -0.18006388, -0.0608395 , -0.78319898,
       -0.48163143, -0.48163143, -0.00473391, -1.37932088, -0.48163143,
       -0.48163143,  0.11449047, -0.60085581, -0.0608395 , -0.0608395 ,
        0.71762557, -0.18006388,  0.35995243,  0.29683364,  0.35995243,
       -0.60085581, -0.23616947, -0.18006388, -0.18006388, -1.2600965 ,
       -1.14087212, -1.19697771, -1.07775333, -2.33311592, -1.14087212])

In [8]:
last5m_norm.shape

(40,)

In [9]:
"""Adding candlesticks"""
def get_normalised_bars_array(bars):
    bars = bars.iloc[-10:, :-1].values.flatten()

    """Normalizing candlesticks"""
    bars = (bars-np.mean(bars))/np.std(bars)
    return bars


"""" Append normalised candlestick bars for all granularity"""
state = np.append(state, get_normalised_bars_array(last5m))
state = np.append(state, get_normalised_bars_array(last1h))
state = np.append(state, get_normalised_bars_array(last1d))
state

array([ 2.21143692e+00,  2.44988568e+00,  6.54506779e-01,  7.17625569e-01,
        7.17625569e-01,  1.97298816e+00,  5.35282399e-01,  1.13841750e+00,
        1.13841750e+00,  1.79064499e+00, -4.18512639e-01, -1.73050681e-01,
       -1.80063880e-01, -6.08394997e-02, -7.83198978e-01, -4.81631429e-01,
       -4.81631429e-01, -4.73390920e-03, -1.37932088e+00, -4.81631429e-01,
       -4.81631429e-01,  1.14490471e-01, -6.00855808e-01, -6.08394997e-02,
       -6.08394997e-02,  7.17625569e-01, -1.80063880e-01,  3.59952429e-01,
        2.96833640e-01,  3.59952429e-01, -6.00855808e-01, -2.36169470e-01,
       -1.80063880e-01, -1.80063880e-01, -1.26009650e+00, -1.14087212e+00,
       -1.19697771e+00, -1.07775333e+00, -2.33311592e+00, -1.14087212e+00,
       -1.41310441e+00, -8.24332730e-01, -1.71691897e+00, -1.25176840e+00,
       -1.25176840e+00, -6.62996719e-01, -2.50474157e+00, -1.25176840e+00,
       -1.23500621e+00,  5.71119003e-01, -1.53882078e+00, -8.90490971e-04,
       -8.90490971e-04,  

In [10]:
state.shape

(120,)

For 5 mins candlesticks bars, you got the normalised candlestick bar of size 40. Similarly, for 1 hour and 1 day bars, you get the candlestick bar of the same shape. So, the final size of the normalised candlesticks bars input feature for 5 mins, 1 hour, and 1 day is 120.

<a id='indicators'></a> 
## Technical indicators

Now, you will create and add technical indicators to the state array. The technical indicators that we will use are:
1. Relative differences between two moving averages
2. Relative Strength Index (RSI) 
3. Momentum
4. Balance of Power (BOP) 
5. Aroon Oscillator

You will use Ta-Lib package to calculate the values of these indicators.

In [11]:
# Create an array to store the value of indicators
tech_ind = np.array([])

"""Relative difference two moving averages"""
sma1 = talib.SMA(last5m['close'], lkbk-1)[-1]
sma2 = talib.SMA(last5m['close'], lkbk-8)[-1]
tech_ind = np.append(tech_ind, (sma1-sma2)/sma2)

"""Relative Strength Index"""
tech_ind = np.append(tech_ind, talib.RSI(
    last5m['close'], lkbk-1)[-1])

"""Momentum"""
tech_ind = np.append(tech_ind, talib.MOM(
    last5m['close'], lkbk-1)[-1])

"""Balance of Power"""
tech_ind = np.append(tech_ind, talib.BOP(last5m['open'],
                                         last5m['high'],
                                         last5m['low'],
                                         last5m['close'])[-1])
"""Aroon Oscillator"""
tech_ind = np.append(tech_ind, talib.AROONOSC(last5m['high'],
                                              last5m['low'],
                                              lkbk-3)[-1])
tech_ind

array([ 5.66049315e-04,  5.58139535e+01,  2.90000000e-01,  4.46927374e-02,
       -7.64705882e+01])

We used five technical indicators, so for each indicator, we get one value. Therefore, the size of this array is 5.

In the previous cell, you have calculated the values of technical indicators for only 5 mins data. Here, similar to that you will calculate values of technical indicators for 5 mins, 1 hour and 1 day data and append to the state array. 

In [12]:
def get_technical_indicators(bars):
    # Create an array to store the value of indicators
    tech_ind = np.array([])

    """Relative difference two moving averages"""
    sma1 = talib.SMA(bars['close'], lkbk-1)[-1]
    sma2 = talib.SMA(bars['close'], lkbk-8)[-1]
    tech_ind = np.append(tech_ind, (sma1-sma2)/sma2)

    """Relative Strength Index"""
    tech_ind = np.append(tech_ind, talib.RSI(
        bars['close'], lkbk-1)[-1])

    """Momentum"""
    tech_ind = np.append(tech_ind, talib.MOM(
        bars['close'], lkbk-1)[-1])

    """Balance of Power"""
    tech_ind = np.append(tech_ind, talib.BOP(bars['open'],
                                             bars['high'],
                                             bars['low'],
                                             bars['close'])[-1])

    """Aroon Oscillator"""
    tech_ind = np.append(tech_ind, talib.AROONOSC(bars['high'],
                                                  bars['low'],
                                                  lkbk-3)[-1])
    return tech_ind

In [13]:
state = np.append(state, get_technical_indicators(last5m))
state = np.append(state, get_technical_indicators(last1h))
state = np.append(state, get_technical_indicators(last1d))
state

array([ 2.21143692e+00,  2.44988568e+00,  6.54506779e-01,  7.17625569e-01,
        7.17625569e-01,  1.97298816e+00,  5.35282399e-01,  1.13841750e+00,
        1.13841750e+00,  1.79064499e+00, -4.18512639e-01, -1.73050681e-01,
       -1.80063880e-01, -6.08394997e-02, -7.83198978e-01, -4.81631429e-01,
       -4.81631429e-01, -4.73390920e-03, -1.37932088e+00, -4.81631429e-01,
       -4.81631429e-01,  1.14490471e-01, -6.00855808e-01, -6.08394997e-02,
       -6.08394997e-02,  7.17625569e-01, -1.80063880e-01,  3.59952429e-01,
        2.96833640e-01,  3.59952429e-01, -6.00855808e-01, -2.36169470e-01,
       -1.80063880e-01, -1.80063880e-01, -1.26009650e+00, -1.14087212e+00,
       -1.19697771e+00, -1.07775333e+00, -2.33311592e+00, -1.14087212e+00,
       -1.41310441e+00, -8.24332730e-01, -1.71691897e+00, -1.25176840e+00,
       -1.25176840e+00, -6.62996719e-01, -2.50474157e+00, -1.25176840e+00,
       -1.23500621e+00,  5.71119003e-01, -1.53882078e+00, -8.90490971e-04,
       -8.90490971e-04,  

In [14]:
state.shape

(135,)

There are 5 technical indicators, and for each granularity of data, you get 5 values. Therefore the size of technical indicators is 15 (5*3). And the total size of the state array is 135, that is (120+15).

<a id='time'></a> 
## Time signature

In this step, you will add the time of the day, day of the week and position to the state array. 

### Time of the day

To calculate the time of the day, we use hours and minutes from the current time. We divide by 24*60 to convert the time into an hour and finally normalise it.

In [15]:
# Extract hours and minutes from current time and map that into float
tm_lst = list(map(float, str(curr_time.time()).split(':')[:2]))

# Calculate time of the day
_time_of_day = (tm_lst[0]*60 + tm_lst[1])/(24*60)
print('Time of the day:', _time_of_day)

Time of the day: 0.4583333333333333


Since we set the current time as (2012, 5, 10, 11, 0, 0). The hour and minutes are 11 and 0 respectively. Then we use the above formula to get the time of the day.

### Day of the week

We use `weekday()` property of datetime to get the day of the week as an integer. Then we divide it by 6 to normalise it.


Syntax: 
```python
DatetimeIndex.weekday()
```
 
Returns:

Integers indicating the day of the week. The day of the week with: <br>
Monday = 0, 
Tuesday = 1,
Wednesday = 2,
Thursday = 3,
Friday = 4,
Saturday = 5,
Sunday = 6

In [16]:
_day_of_week = curr_time.weekday()/6
print('Day of the Week:', _day_of_week)

Day of the Week: 0.5


Since we set the current time as (2012, 5, 10, 11, 0, 0), which falls on Thursday, therefore the day of the week is 0.5. 

### Position

In [17]:
# The position is updated using update_postion() function of Game class. 
# For the illustration purpose, we have manually set to 1.
position = 1

In [18]:
state = np.append(state, _time_of_day)
state = np.append(state, _day_of_week)
state = np.append(state, position)

In [19]:
state

array([ 2.21143692e+00,  2.44988568e+00,  6.54506779e-01,  7.17625569e-01,
        7.17625569e-01,  1.97298816e+00,  5.35282399e-01,  1.13841750e+00,
        1.13841750e+00,  1.79064499e+00, -4.18512639e-01, -1.73050681e-01,
       -1.80063880e-01, -6.08394997e-02, -7.83198978e-01, -4.81631429e-01,
       -4.81631429e-01, -4.73390920e-03, -1.37932088e+00, -4.81631429e-01,
       -4.81631429e-01,  1.14490471e-01, -6.00855808e-01, -6.08394997e-02,
       -6.08394997e-02,  7.17625569e-01, -1.80063880e-01,  3.59952429e-01,
        2.96833640e-01,  3.59952429e-01, -6.00855808e-01, -2.36169470e-01,
       -1.80063880e-01, -1.80063880e-01, -1.26009650e+00, -1.14087212e+00,
       -1.19697771e+00, -1.07775333e+00, -2.33311592e+00, -1.14087212e+00,
       -1.41310441e+00, -8.24332730e-01, -1.71691897e+00, -1.25176840e+00,
       -1.25176840e+00, -6.62996719e-01, -2.50474157e+00, -1.25176840e+00,
       -1.23500621e+00,  5.71119003e-01, -1.53882078e+00, -8.90490971e-04,
       -8.90490971e-04,  

In [20]:
state.shape

(138,)

Finally, we added 3 more features to the state array, that is the 'day of the week', 'time of the day' and 'position'. And for each of these features, we get one value. Therefore the final size of the state array is 138, that is (135+3). We pass this state array to the neural networks to predict the action that is, buy, sell, or hold. 

Till now, you have learned to create a state array, update position, and design a reward system. In the next notebook, you will learn to put it all together in the Game class.<br><br>